<a href="https://colab.research.google.com/github/Erike-Simon/Airbnb-Project/blob/main/Projeto_final_aibnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SISTEMA DE LOCALIZAÇÃO DE HOSPEDAGENS PRÓXIMAS A PONTOS TURÍSTICOS NA CIDADE DO RECIFE**

---

Projeto final da disciplina Análise e Visualização de Dados do CESAR School Recife.

**Alunos:** *Erike Simon e Jadeilson Santos*

O presente projeto visa explorar e desenvolver um dashboard no streamlit através de um dataset airbnb fornecido na disciplina de Análise e Visualização de Dados. Além deste, outros datasets tbm serão utilizados para localizações de pontos turísticos na cidade do Recife (teatros, museus e mercados). Estes últimos foram encontrados no [portal de dados](http://dados.recife.pe.gov.br/pt_BR/) da prefeitura do Recife.




**Descrever projeto e dataset**



# Configurações iniciais

In [ ]:
# import das bibliotecas
import numpy as np
import pandas as pd
import re
import warnings
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
# from ydata_profiling import ProfileReport

In [ ]:
# Ignorar os avisos do tipo SettingWithCopyWarning
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/dataset_airbnb-scraper_rec.csv')
df_mercado = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/mercados.csv')
df_museu = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/museu-1.csv', sep=';')
df_teatro = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/teatros.csv')

In [ ]:
# dataset contendo apenas as colunas que serão utilizadas
df1 = df[['address','location/lat','location/lng','name','numberOfGuests','pricing/nights','pricing/rate/amount','pricing/rate/amountFormatted',
          'pricing/rateType', 'pricing/totalPrice/accommodation','pricing/totalPrice/amount','pricing/totalPrice/amountFormatted','pricing/totalPrice/cleaningFee',
          'pricing/totalPrice/currency','pricing/totalPrice/guestFee','primaryHost/badges/0','primaryHost/badges/1','roomType','stars']]

In [ ]:
df1.drop_duplicates()

In [ ]:
df1.isnull().sum()

address                                 0
location/lat                            0
location/lng                            0
name                                    0
numberOfGuests                          0
pricing/nights                          0
pricing/rate/amount                     0
pricing/rate/amountFormatted            0
pricing/rateType                        0
pricing/totalPrice/accommodation        0
pricing/totalPrice/amount               0
pricing/totalPrice/amountFormatted      0
pricing/totalPrice/cleaningFee        177
pricing/totalPrice/currency             0
pricing/totalPrice/guestFee            60
primaryHost/badges/0                    7
primaryHost/badges/1                   48
roomType                                0
stars                                 125
dtype: int64

## Funções




In [ ]:
pd.set_option('display.max_rows', None) #para mostras todas as linhas nas operações nas colunas

Função para padronização dos endereços

In [ ]:
def stand_address(local):
    local = local.lower()  # Converter para minúsculas
    local = re.sub(r'[^\w\s]', ' ', local)  # Remover pontuações
    local = re.sub(r'\s+', ' ', local)  # Remover espaços em branco extras

    # Dicionário de palavras-chave e valores de padronização
    chave_map = {
        'porto de galinhas': 'Porto de Galinhas', 'muro alto': 'Muro Alto', 'enseada dos corais': 'Enseada dos Corais', 'recife': 'Recife', 'boa viagem': 'Boa Viagem', 'sao jose': 'São José',
        'boa vista': 'Boa Vista', 'madalena': 'Madalena', 'encruzilhada': 'Encruzilhada', 'casa amarela': 'Casa Amarela', 'cordeiro': 'Cordeiro', 'casa forte': 'Casa Forte', 'pina': 'Pina',
        'parnamirim': 'Parnamirim', 'santo antônio': 'Santo Antônio', 'várzea': 'Várzea', 'apipucos': 'Apipucos', 'graças': 'Graças',  'cabo de santo agostinho' : 'Cabo de Santo Agostinho',
        'soledade': 'Soledade', 'santo amaro': 'Santo Amaro', 'ipojuca' : 'Ipojuca', 'olinda' : 'Olinda', 'piedade' : 'Piedade', 'carmo' : 'Carmo', 'aldeia dos camarás' : 'Aldeia dos Camarás',
        'paiva' : 'Paiva', 'candeias' : 'Candeias', 'barra de jangada' : 'Barra de Jangada', 'brasília teimosa' : 'Brasília Teimosa', 'centro' : 'Centro',  'torre' : 'Torre', 'paulista' : 'Paulista',
        'santana' : 'Santana', 'sítio novo' : 'Sítio Novo', 'rio doce' : 'Rio Doce', 'derby' : 'Derby', 'casa caiada' : 'Casa Caiada', 'engenho do meio' : 'Engenho do Meio', 'piedade' : 'Piedade',
        'jaboatão dos guararapes' : 'Jaboatão dos Guararapes', 'espinheiro' : 'Espinheiro', 'rosarinho' : 'Rosarinho', 'curado' : 'Curado', 'água fria' : 'Água Fria', 'estância' : 'Estância',
        'tamarineira' : 'Tamarineira', 'jardim atlântico' : 'Jardim Atrântico', 'bonsucesso' : 'Bonsucesso', 'aflitos' : 'Aflitos', 'ipsep' : 'Ipsep', 'mangueira' : 'Mangueira', 'ibura' : 'Ibura',
        'ipojuga' : 'Ipojuca', 'dois irmãos' : 'Dois Irmãos', 'ilha do retiro' : 'Ilha do Retiro', 'loteamento merepe' : 'Loteamento Merepe', 'varadouro' : 'Varadouro', 'iputinga' : 'Iputinga',
        'dois carneiros' : 'Dois Carneiros', 'bairro novo' : 'Bairro Novo', 'socorro' : 'Socorro', 'bultrins' : 'Bultrins', 'ouro preto' : 'Ouro Preto', 'zumbi' : 'Zumbi', 'camaragibe' : 'Camaragibe',
        'loteamento mata da camboa' : 'Loteamento Mata da Camboa','campo grande' : 'Campo Grande'
    }

    # Verificar cada palavra-chave no endereço e retornar o valor correspondente
    for chave, valor in chave_map.items():
        if chave in local:
            return valor

    # Caso não corresponda a uma palavra-chave, retorne o local original
    return local

In [ ]:
# Criação das colunas com endereços padronizados
df1['stand address'] = df1['address'].apply(stand_address)
df_mercado['stand address'] = df_mercado['bairro'].apply(stand_address)
df_museu['stand address'] = df_museu['bairro'].apply(stand_address)
df_teatro['stand address'] = df_teatro['bairro'].apply(stand_address)

## Ajustes e criação de novas colunas

Filtrando os bairros do Recife e criando um novo dataset

In [ ]:
# Para filtrar os bairros do Recife
bairros_rec = ['Boa Viagem', 'Água Fria', 'Recife', 'Rosarinho', 'Espinheiro', 'Iputinga', 'Santana', 'Zumbi',
               'Brasília Teimosa', 'Boa Vista', 'Pina', 'Parnamirim', 'Madalena', 'Santo Amaro', 'Campo Grande',
               'Centro', 'Derby', 'Cordeiro', 'Graças', 'Casa Forte', 'Casa Amarela', 'Ilha do Retiro', 'Curado',
               'Várzea', 'Estância', 'Tamarineira', 'Apipucos', 'Torre', 'Ipsep', 'Aflitos', 'Mangueira', 'Ibura',
               'Dois Irmãos', 'Soledade', 'Encruzilhada', 'Engenho do Meio']

In [ ]:
# Novo DF contendo apenas os bairros do Recife
df_rec = df1[df1['stand address'].isin(bairros_rec)]

Criação de uma nova coluna com classificação de praia

In [ ]:
# Criando uma nova coluna e atribuindo valor default
df_rec['praia'] = 0

# localiza as linhas que possuem os valores desejados na coluna 'stand address' e pela coluna 'praia' e atribui na coluna 'praia' o valor 1 para estas linhas
df_rec.loc[df_rec['stand address'].isin(['Boa Viagem', 'Pina', 'Brasília Teimosa']), 'praia'] = 1

Crianção de colunas para preços totais por noite e preços totais por hóspedes para cada noite

In [ ]:
# Criando duas novas colunas, uma com o preço médio por hóspede 'price per guest' e outra com o preço médio por noite 'price per night'
df_rec['price per guest'] = round((df_rec['pricing/totalPrice/amount'] / 3) / df_rec['numberOfGuests'], 3)
df_rec['price per night'] = round(df_rec['pricing/totalPrice/amount'] / 3, 3)

Criação da coluna 'price per night' normalizada inversa

In [ ]:
df_rec['norm price per guest'] = round(1 - (df_rec['price per guest']/df_rec['price per guest'].max()), 3)

Criação da coluna 'reviews numerical' com os dados de reviews ajustados

In [ ]:
# Substitui oas vírgulas por um espaço vazio
df_rec['primaryHost/badges/0'] = df_rec['primaryHost/badges/0'].str.replace(',', '')

In [ ]:
# extrai da coluna 'primaryHost/badges/0'  apenas os valores numéricos, transformando-os para float e criando uma nova coluna com esses valores
df_rec['reviews numerical'] = df_rec['primaryHost/badges/0'].str.extract(r'(\d+)', expand=False).astype(float)
# Preenche os valores nulos com 0
df_rec['reviews numerical'].fillna(0, inplace=True)

Substituindo dados nulos na coluna 'stars' por zero

In [ ]:
df_rec['stars'] = df_rec['stars'].fillna(0)

Criação da coluna 'score' para armazenar a pontuação geral de cada hospedagem


*   Equação da pontuação: ***S = (PHN) x peso + ST x peso + RW x peso***



onde PHN é a coluna 'norm price per guest', ST é a coluna 'stars' e RW é a coluna 'reviews numerical'.

Nota-se no rankeamento que nem sempre as melhores hospedagens são as mais caras ou mais bem avaliadas por 'stars'. É uma questão de balanço entre algumas variáveis. Esse será justamente o critério chave na busca por hospedagens a serem exibidas para o cliente, de maneira que ele escolha as melhores rankeadas e mais próximas dos pontos turísticos de interesse.

In [ ]:
# definindo os pesos
peso_PHN = 0.6
peso_ST = 0.3
peso_RW = 0.1

# cálculo de acordo com a equação da descrição
df_rec['score'] = round((df_rec['norm price per guest']*peso_PHN +
                   df_rec['stars']*peso_ST + df_rec['reviews numerical']*peso_RW), 3)

# ordenando os valores decrescentes
df_rec = df_rec.sort_values('score', ascending=False)

# Exploração e gráficos

## Mapa das localizações

In [ ]:
prices = df_rec['price per night']
# Define os máximos e mínimos tamanhos para os marcadores
max_size = 12
min_size = 5

# Escala os preços para ajustar a um tamanho proporcional
scaled_sizes = [
    (price - min(prices)) / (max(prices) - min(prices)) * (max_size - min_size) + min_size
    for price in prices
]

# Criação dos gráficos (traces)
trace1 = go.Scattermapbox(
    lat = df_rec['location/lat'],
    lon = df_rec['location/lng'],
    mode='markers',
    marker=dict(
        size=scaled_sizes,
        sizemode='diameter',
        sizeref=0.6,  # Controla a escala dos tamanhos dos marcadores
        color='blue',
        opacity=0.6
    ),
    hovertext=[
        f"{nome}<br>Preço por noite: {preco} R$<br>Hóspedes: {hosp}"
        for nome, preco, hosp in zip(df_rec['name'], df_rec['price per night'], df_rec['numberOfGuests'])
    ],
    name='Hospedagens'
)

trace2 = go.Scattermapbox(
    lat = df_museu['latitude'],
    lon = df_museu['longitude'],
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        opacity=0.7
    ),
    hovertext=[
        f"{nome}<br>Site: <a href='{site}'>{site}</a>"
        for nome, site in zip(df_museu['nome'], df_museu['site'])
    ],
    name='Museus'
)

trace3 = go.Scattermapbox(
    lat = df_mercado['latitude'],
    lon = df_mercado['longitude'],
    mode='markers',
    marker=dict(
        size=10,
        color='green',
        opacity=0.7
    ),
    hovertext=df_mercado['nome'],
    name='Mercado'
)

trace4 = go.Scattermapbox(
    lat = df_teatro['latitude'],
    lon = df_teatro['longitude'],
    mode='markers',
    marker=dict(
        size=10,
        color='purple',
        opacity=0.8
    ),
    hovertext=[
        f"{nome}<br>Fone: {fone}"
        for nome, fone in zip(df_teatro['nome'], df_teatro['Telefone'])
    ],
    name='Teatros'
)

layout = go.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style = 'open-street-map',
        center=dict(lat=df_rec['location/lat'].mean(), lon=df_rec['location/lng'].mean()),
        zoom=10.7
    ),
    showlegend=True
)

traces=[trace1, trace2, trace3, trace4]

# Cria a figura com o scattermap e o layout
fig = go.Figure(data=traces, layout=layout)

fig.show()


O mapa destaca a localização geográfica das hospedagens da Airbnb em Recife, permitindo que os usuários tenham uma visão geral das opções disponíveis em diferentes áreas da cidade. Essa visualização é valiosa para viajantes que desejam escolher acomodações próximas a pontos turísticos específicos, facilitando o planejamento de suas atividades durante a estadia.

A sobreposição dos pontos turísticos, como museus, mercados e teatros, ao mapa das hospedagens é uma informação valiosa para os viajantes. Isso permite que eles identifiquem rapidamente quais hospedagens estão próximas aos pontos de interesse que desejam visitar. A proximidade dessas atrações pode influenciar a escolha da acomodação, pois pode economizar tempo e custos de transporte durante a viagem.

A escala dos tamanhos dos marcadores com base nos preços por noite das acomodações é uma maneira inteligente de visualizar a relação entre preço e localização. Através dos pontos de diferentes tamanhos, os usuários podem identificar áreas onde as acomodações são mais acessíveis ou mais luxuosas. Essa informação é especialmente útil para viajantes que buscam acomodações que atendam ao seu orçamento ou preferências de luxo.

## Mediana de preço por noite e por hóspede




Análise

In [ ]:
# Localizações mais comuns
com_address = df_rec['stand address'].value_counts().head(5)
com_address

Boa Viagem    89
Recife        64
Boa Vista      6
Graças         5
Várzea         4
Name: stand address, dtype: int64

A análise dos dados revela que as localizações mais comuns para hospedagens em Recife são Boa Viagem, Recife, Boa Vista, Graças e Várzea. Boa Viagem se destaca com o maior número de hospedagens, provavelmente devido à sua popularidade devido à bela praia e infraestrutura turística. Recife, como a capital, também oferece muitas opções de acomodação. Outras áreas menos turísticas, como Boa Vista, Graças e Várzea, também possuem algumas opções de hospedagem para aqueles que buscam experiências mais autênticas e tranquilas.

In [ ]:
median_night_prices = round(df_rec.loc[df_rec['stand address'].isin(com_address.index),
                                       'price per night'].groupby(df_rec['stand address']).median().sort_values(ascending=False), 3)
median_night_prices

stand address
Boa Viagem    360.667
Graças        326.000
Recife        286.667
Boa Vista     109.333
Várzea         82.833
Name: price per night, dtype: float64

A análise dos preços medianos por noite revela que Boa Viagem tem as hospedagens mais caras, seguida pelas Graças e Recife. Boa Vista apresenta preços intermediários, enquanto Várzea é a localização mais econômica. Essas informações ajudam os viajantes a escolherem acomodações que atendam às suas preferências e orçamentos durante a estadia em Recife.

In [ ]:
median_night_prices_guest = round(df_rec.loc[df_rec['stand address'].isin(com_address.index), 'price per guest'].groupby(df_rec['stand address']).median(), 3)
median_night_prices_guest

stand address
Boa Viagem    88.778
Boa Vista     49.917
Graças        81.500
Recife        76.362
Várzea        52.750
Name: price per guest, dtype: float64

A análise dos preços medianos por hóspede para as localizações mais comuns em Recife revela que Boa Viagem apresenta os valores mais altos, seguida por Graças e Recife. Boa Vista tem preços medianos intermediários, enquanto Várzea oferece os preços mais baixos por hóspede. Essas informações auxiliam os viajantes a entenderem os custos de hospedagem por pessoa em diferentes áreas da cidade.

In [ ]:
sorted_prices_guest = median_night_prices_guest.reindex(median_night_prices.index)

Gráfico

In [ ]:
# Plot do gráfico
trace1 = go.Bar(
        x=median_night_prices.index, y=median_night_prices.values,
        name='Preço por noite',
        marker_color='rgb(27, 158, 119)',
        text=median_night_prices.values, textposition='auto'
        )

trace2 = go.Bar(
        x=sorted_prices_guest.index, y=sorted_prices_guest.values,
        name='Preço por hóspede',
        marker_color='rgb(117,112,179)',
        text=sorted_prices_guest.values, textposition='auto'
        )

layout = go.Layout(
    title='Mediana de preço por noite e por hóspede para cada localização', title_x=0.45,
    yaxis=dict(title='Preço Por Noite (R$)', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black'),
    xaxis=dict(title='Locais', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black'),
    legend=dict(x=0.85, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    barmode='group', bargap=0.15, bargroupgap=0.1, plot_bgcolor='white', height=600, width=1300
    )

traces = [trace1, trace2]

# Create the figure with all the boxplot traces
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

O gráfico apresenta a mediana dos preços por noite e por hóspede para cada localização em Recife. Ele utiliza barras para comparar os valores em diferentes áreas da cidade. A barra verde representa o preço mediano por noite, enquanto a barra roxa representa o preço mediano por hóspede. Esse gráfico é eficaz para os viajantes entenderem as diferenças de custo entre as localizações e planejarem suas estadias de acordo com suas preferências e orçamento.

In [ ]:
# Proporção de preço pago por hospede
rate_night_price_guest = round((sorted_prices_guest / median_night_prices)*100, 2)
rate_night_price_guest

stand address
Boa Viagem    24.61
Graças        25.00
Recife        26.64
Boa Vista     45.66
Várzea        63.68
dtype: float64

Nota-se que as hospedagens do bairro das Graças possui uma proporção de preço pago por hóspede maior do que o bairro do Recife. Isso pode ser influenciado pela quantidade de hosédagens em cada um dos bairros. Recife possui 64 e Graças apenas 6. Note também que Graças possui a segunda maior média de preços por noite e também por hóspedes. É possível que a medida que o bairro possua mais locais disponíveis para locação, a discrepância de preços pago por noite e por hóspede diminua.

### Mediana de preços por noite e por hóspedes para cada tipo de hospedagem para os bairros mais comuns

Análise

In [ ]:
median_type_prices = round(df_rec.loc[df_rec['stand address'].isin(com_address.index)].groupby(
                           df_rec['roomType'])['price per night'].median().sort_values(ascending=False), 3)
median_type_prices

roomType
Private room in condo                 25000.000
Room in hotel                          9999.000
Shared room in rental unit             3423.667
Ranch                                  1369.333
Entire home                             620.000
Entire rental unit                      383.333
Private room in home                    373.167
Entire condo                            326.000
Entire serviced apartment               313.000
Entire loft                             312.000
Private room in serviced apartment      258.667
Private room in rental unit             106.667
Private room in guest suite              74.333
Entire guesthouse                        68.333
Shared room in bed and breakfast         62.667
Shared room in hostel                    56.000
Name: price per night, dtype: float64

Essa informação apresenta a mediana dos preços por tipo de quarto para as localizações mais comuns em Recife. O resultado mostra diferentes categorias de quartos, como quartos privados em condomínio, quartos em hotéis, quartos compartilhados em unidades de aluguel, entre outros. Os preços variam entre as categorias, o que pode ser útil para viajantes que desejam selecionar o tipo de acomodação mais adequado para suas necessidades e orçamento.

In [ ]:
median_type_guest = round(df_rec.loc[df_rec['stand address'].isin(com_address.index)].groupby(
                           df_rec['roomType'])['price per guest'].median().reindex(median_type_prices.index), 3)
median_type_guest

roomType
Private room in condo                 8333.333
Room in hotel                         2499.750
Shared room in rental unit             228.244
Ranch                                  171.167
Entire home                            124.000
Entire rental unit                      83.333
Private room in home                   116.417
Entire condo                            76.067
Entire serviced apartment               88.917
Entire loft                            156.000
Private room in serviced apartment     107.611
Private room in rental unit             57.000
Private room in guest suite             37.167
Entire guesthouse                       68.333
Shared room in bed and breakfast         6.963
Shared room in hostel                    3.733
Name: price per guest, dtype: float64

O código apresenta a mediana dos preços por hóspede para diferentes categorias de quartos nas localizações mais comuns em Recife. Os valores mostram quanto cada tipo de quarto custa em média por hóspede, permitindo que os viajantes entendam o custo relativo de diferentes opções de acomodação.

Gráfico 1

In [ ]:
# Horizontal bar
trace1 = go.Bar(
        x=median_type_prices.values, y=median_type_prices.index,
        name='Preço médio',
        marker_color='rgb(27, 158, 119)', orientation='h',
        text=median_type_prices.values, textposition='outside'
        )

trace2 = go.Bar(
        x=median_type_guest.values, y=median_type_guest.index,
        name='Preço médio por hóspede',
        marker_color='rgb(117,112,179)', orientation='h',
        text=median_type_guest.values, textposition='outside'
        )

layout = go.Layout(
    title='Mediana de preços por noite e por hóspede para cada tipo de hospedagem', title_x=0.45,
    yaxis=dict(title='Tipo de Hospedagem', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black'),
    xaxis=dict(title='Preço Médio Por Noite (R$)', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black', type='linear',  # Altera o tipo do eixo para linear
               dtick=2500),
    legend=dict(x=0.60, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    barmode='group', bargap=0.15, bargroupgap=0.1, plot_bgcolor='white', height=800, width=1300
    )

traces = [trace1, trace2]

# Create the figure with all the boxplot traces
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

Análise

In [ ]:
# tipos mais comuns dos bairros mais comuns
com_type = df_rec.loc[df_rec['stand address'].isin(com_address.index)]['roomType'].value_counts().head(5)
com_type

Entire rental unit             77
Entire serviced apartment      33
Private room in rental unit    21
Entire condo                   17
Entire loft                     5
Name: roomType, dtype: int64

O código apresenta a mediana dos preços por hóspede para diferentes categorias de quartos nas localizações mais comuns em Recife. Os valores mostram quanto cada tipo de quarto custa em média por hóspede, permitindo que os viajantes entendam o custo relativo de diferentes opções de acomodação.

In [ ]:
median_com_type_night = median_type_prices.loc[com_type.index].sort_values(ascending=False)
median_com_type_night

Entire rental unit             383.333
Entire condo                   326.000
Entire serviced apartment      313.000
Entire loft                    312.000
Private room in rental unit    106.667
Name: price per night, dtype: float64

O código apresenta a mediana dos preços por noite para as categorias de quartos mais comuns em Recife. Os resultados mostram as cinco categorias de quartos mais frequentes, classificadas em ordem decrescente de preço mediano. Essas informações auxiliam os viajantes a entenderem a faixa de preços associada a cada tipo de acomodação nas áreas mais populares da cidade.

In [ ]:
median_com_type_guest = median_type_guest.loc[com_type.index].reindex(median_com_type_night.index)
median_com_type_guest

Entire rental unit              83.333
Entire condo                    76.067
Entire serviced apartment       88.917
Entire loft                    156.000
Private room in rental unit     57.000
Name: price per guest, dtype: float64

O código apresenta a mediana dos preços por hóspede para as categorias de quartos mais comuns em Recife, correspondentes aos tipos de quartos com os preços médios mais altos por noite. Os valores mostram os preços medianos por hóspede para "Unidade de aluguel completa", "Condomínio inteiro", "Apartamento inteiro com serviços", "Loft inteiro" e "Quarto privado em unidade de aluguel".

Gráfico 2

In [ ]:
# Horizontal bar
trace1 = go.Bar(
        x=median_com_type_night.values, y=median_com_type_night.index,
        name='Preço médio',
        marker_color='rgb(27, 158, 119)', orientation='h',
        text=median_com_type_night.values, textposition='outside'
        )

trace2 = go.Bar(
        x=median_com_type_guest.values, y=median_com_type_guest.index,
        name='Preço médio por hóspede',
        marker_color='rgb(117,112,179)', orientation='h',
        text=median_com_type_guest.values, textposition='outside'
        )

layout = go.Layout(
    title='Mediana de preços por noite e por hóspede para hospedagens mais comuns', title_x=0.45,
    yaxis=dict(title='Tipo de Hospedagem', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black'),
    xaxis=dict(title='Preço Médio Por Noite (R$)', titlefont_size=16, tickfont_size=14,
               showline=True, linewidth=1, linecolor='black', type='linear',  # Altera o tipo do eixo para linear
               dtick=2500),
    legend=dict(x=0.60, y=1.0, bgcolor='rgba(255, 255, 255, 0)', bordercolor='rgba(255, 255, 255, 0)'),
    barmode='group', bargap=0.15, bargroupgap=0.1, plot_bgcolor='white', height=600, width=1100
    )

traces = [trace1, trace2]

# Create the figure with all the boxplot traces
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

In [ ]:
# Taxas de preços pagos por hóspede e por noite para os tipos mais comuns de hospedagens nos bairros mais comuns
rate_com_type = round((median_com_type_guest / median_com_type_night)*100, 2).sort_values(ascending=False)
rate_com_type

Private room in rental unit    53.44
Entire loft                    50.00
Entire serviced apartment      28.41
Entire condo                   23.33
Entire rental unit             21.74
dtype: float64

Observa-se que apesar do preço por noite de *Private room in rental unit* ser o menor pelo gráfico acima, a taxa de preço paga por hóspede é maior em relação aos demais tipos. Por outro lado, *Entire rental unit* possui a menor proporção de preço pago por hóspede, pois mesmo tendo o maior valor de preço por noite, possui também um dos menores preços pagos por hóspedes. Esse comportamento pode ter relação com quantidade de hóspedes que cada tipo de hospedagem comporta.

### Distribuição e média do número de hóspedes por tipos de hospedagens mais comuns para os bairros mais comuns

Análise

In [ ]:
# dataframe para o strip plot
df_strip = df_rec.loc[df_rec['stand address'].isin(com_address.index) & df_rec['roomType'].isin(com_type.index)].reset_index()

# Crie uma lista com as categorias na ordem do plot
categories_order = df_strip['roomType'].unique()

# Para calcular a média de hóspedes por tipo de hospedagem
avg_guest = df_strip.groupby('roomType')['numberOfGuests'].median()

# Reindexa os índices para que fiquem os mesmos do plot do strip
avg_guest = avg_guest.reindex(categories_order)

O código realiza uma análise da distribuição e da média do número de hóspedes para os tipos de hospedagens mais comuns nas localizações mais frequentes em Recife. Através de um strip plot, é possível visualizar como o número de hóspedes varia para cada categoria de quarto. Além disso, o código calcula a média de hóspedes por tipo de hospedagem, fornecendo informações importantes para os viajantes entenderem a capacidade média de cada tipo de acomodação.

Gráfico

In [ ]:
# Cria o strip plot
fig = px.strip(df_strip, x='roomType', y='numberOfGuests',
               title='Distribuição de preços por noite por quantidade de hóspedes',
               template="simple_white", color_discrete_sequence=px.colors.qualitative.Dark2,
               stripmode='overlay')  # Contorno dos pontos

# Cria um scatter de tendência
avg_trace = go.Scatter(
    x=avg_guest.index,
    y=avg_guest.values,
    mode='lines',
    name='Média de preço',
    line=dict(color='black', width=1.5, dash='dash')
)

# Adiciona a figura o scatter de tendência
fig.add_trace(avg_trace)

fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='black')))

fig.update_xaxes(title='Tipo de Hospedagem', titlefont_size=16, tickfont_size=14)
fig.update_yaxes(title='Quantidade de Hóspedes', titlefont_size=16, tickfont_size=14)

fig.update_layout(title=dict(x=0.5), height=600, width=1200)

fig.show()

In [ ]:
avg_guest.sort_values(ascending=False)

roomType
Entire rental unit             5.0
Entire condo                   5.0
Entire serviced apartment      4.0
Entire loft                    3.0
Private room in rental unit    2.0
Name: numberOfGuests, dtype: float64

Tomando o gráfico anterior de preços por noite e por hóspedes para os tipos de hospedagens, observa-se que *Entire rental unit* possui uma média de hóspedes maior em relação a *Private room in rental unit*, obviamente e também uma maior distribuição para a quantidade de hóspedes. Por isso, os preços são afetados por essas quantidades, fazendo com que *Entire rental unit*, por exemplo, tenha uma menor taxa de preço pago por hóspede para cada noite em relação a *Private room in rental unit*.